## TRCWA

TODO explain what this is and where it comes from

In [ ]:
### Imports
%load_ext autoreload
%autoreload 2

# Append main folder
import sys
sys.path.append("..")

import nidn
import torch

In [ ]:
# Start with the default config
cfg = nidn.load_default_cfg()

In [ ]:
# Titanium Oxide Uniform
# Set grid specifics
cfg.Nx = 15
cfg.Ny = 15
cfg.N_layers = 1
cfg.N_freq = 32
cfg.TRCWA_L_grid = [[0.1,0.0],[0.0,0.1]]
cfg.TRCWA_NG = 11
cfg.TRCWA_PER_LAYER_THICKNESS = [1.0]
cfg.freq_distribution = "linear"

# Specify your desired range of wavelengths
cfg.physical_wavelength_range[0] = 1e-6
cfg.physical_wavelength_range[1] = 1e-5

# Determine target frequencies (in TRCWA units)
cfg.target_frequencies = nidn.compute_target_frequencies(
    cfg.physical_wavelength_range[0],
    cfg.physical_wavelength_range[1],
    cfg.N_freq,
    cfg.freq_distribution
)

In [ ]:
# Init eps_grid
eps_grid = torch.zeros(cfg.Nx,cfg.Ny,cfg.N_layers,cfg.N_freq,dtype=torch.cfloat)

layer_builder = nidn.LayerBuilder(cfg)

In [ ]:
# Uniform Titanium oxide
eps_grid[:,:,0,:] = layer_builder.build_uniform_layer("titanium_oxide")
eps_grid[:,:,1,:] = layer_builder.build_uniform_layer("germanium")
eps_grid[:,:,2,:] = layer_builder.build_uniform_layer("tantalum_pentoxide")
# eps_grid[:,:,0,:] = layer_builder.build_circle_layer("tantalum_pentoxide","titanium_oxide")

In [ ]:
# Plot the abs values of the produced grid
nidn.plot_epsilon_grid(eps_grid,cfg)

In [ ]:
# Print run config
nidn.print_cfg(cfg)

In [ ]:
# Compute spectrum for this configuration
R,T = nidn.compute_spectrum(eps_grid,cfg)

In [ ]:
nidn.plot_spectrum(cfg,R,T)

In [ ]:
print("R = [")
[print(f"{r.item():.8f}",end=",") for r in R]
print("]")
print("T = [")
[print(f"{t.item():.8f}",end=",") for t in T]
print("]")

In [ ]:
# Set grid specifics
cfg.Nx = 1
cfg.Ny = 1
cfg.N_layers = 25
cfg.N_freq = 40
cfg.TRCWA_L_grid = [[1.,0.0],[0.0,1.]]
cfg.TRCWA_NG = 11

# Specify your desired range of wavelengths
cfg.physical_wavelength_range[0] = (1.3)*1e-6
cfg.physical_wavelength_range[1] = (1.7)*1e-6

# Determine target frequencies (in TRCWA units)
cfg.target_frequencies = nidn.compute_target_frequencies(
    cfg.physical_wavelength_range[0],
    cfg.physical_wavelength_range[1],
    cfg.N_freq,
    cfg.freq_distribution
)

In [ ]:
# Init eps_grid
eps_grid = torch.zeros(cfg.Nx,cfg.Ny,cfg.N_layers,cfg.N_freq,dtype=torch.cfloat)

layer_builder = nidn.LayerBuilder(cfg)

In [ ]:
thicknesses = []

n_2 = 2.2
n_1 = 1.5
eps_1 = n_1**2
eps_2 = n_2**2

lam0 = 1.50
d_1 = lam0/(4*n_1)
d_2 = lam0/(4*n_2)
d_defect = lam0/(2*n_1)

# Top DBR
for i in range(6):
    eps_grid[:,:,2*i,:] = eps_1
    thicknesses.append(d_1)
    eps_grid[:,:,2*i+1,:] = eps_2
    thicknesses.append(d_2)
    
# Defect 
eps_grid[:,:,12,:] = eps_1
thicknesses.append(d_defect)

# Bottom DBR
for i in range(6,12):
    eps_grid[:,:,2*i+1,:] = eps_2
    thicknesses.append(d_2)
    eps_grid[:,:,2*i+2,:] = eps_1
    thicknesses.append(d_1)

thicknesses.reverse()
cfg.TRCWA_PER_LAYER_THICKNESS = thicknesses
    
# eps_grid[:,:,0,:] = layer_builder.build_uniform_layer("titanium_oxide")
# eps_grid[:,:,0,:] = layer_builder.build_squared_layer("germanium","tantalum_pentoxide")
#eps_grid[:,:,1,:] = layer_builder.build_uniform_layer("aluminium_nitride")
#eps_grid[:,:,2,:] = layer_builder.build_squared_layer("titanium_oxide","germanium")

In [ ]:
d_1

In [ ]:
d_2

In [ ]:
d_defect

In [ ]:
# Plot the abs values of the produced grid
nidn.plot_epsilon_grid(eps_grid,cfg)

In [ ]:
# Compute spectrum for this configuration
R,T = nidn.compute_spectrum(eps_grid,cfg)

In [ ]:
nidn.plot_spectrum(cfg,R,T)

In [ ]:
print("R = [")
[print(f"{r.item():.8f}",end=",") for r in R]
print("]")
print("T = [")
[print(f"{t.item():.8f}",end=",") for t in T]
print("]")